In [11]:
#!pip install -q keras==2.3.1
#!pip install -q tensorflow-estimator==2.1
#!pip install imutils

In [3]:
# import the necessary packages
import os
# initialize the path to the input directory containing our dataset
# of images
DATASET_PATH = "C:/Users/97150/Desktop/Uno_card_color_shape_detection/Unocards_data"
# initialize the class labels in the dataset
CLASSES = ["Card_0","Card_1","Card_2","Card_3","Card_4","Card_5","Card_6","Card_7","Card_8","Card_9","Draw_2+","Draw_4+","Reverse","Skip_Card","Wild_Card"]

In [2]:
# define the size of the training, validation (which comes from the
# train split), and testing splits, respectively
TRAIN_SPLIT = 0.80
VAL_SPLIT = 0.1
TEST_SPLIT = 0.20

In [3]:
# define the minimum learning rate, maximum learning rate, batch size,
# step size, CLR method, and number of epochs
MIN_LR = 1e-6
MAX_LR = 1e-4
BATCH_SIZE = 32
STEP_SIZE = 8
CLR_METHOD = "triangular"
NUM_EPOCHS = 30

In [4]:
# set the path to the serialized model after training
MODEL_PATH = os.path.sep.join(["output", "unocard.model"])
# define the path to the output learning rate finder plot, training
# history plot and cyclical learning rate plot
LRFIND_PLOT_PATH = os.path.sep.join(["output", "lrfind_plot.png"])
TRAINING_PLOT_PATH = os.path.sep.join(["output", "training_plot.png"])
CLR_PLOT_PATH = os.path.sep.join(["output", "clr_plot.png"])

In [5]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import sys
import os



Using TensorFlow backend.


In [6]:
# grab the paths to all images in our dataset directory and initialize
# our lists of images and class labels
print("[INFO] loading images...")
imagePaths = list(paths.list_images(DATASET_PATH))
data = []
labels = []

[INFO] loading images...


In [7]:
#print(imagePaths)

In [8]:
# loop over the image paths
for imagePath in imagePaths:
    # extract the class label
    label = imagePath.split(os.path.sep)[-2]
    # load the image, convert it to RGB channel ordering, and resize
    # it to be a fixed 224x224 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
# convert the data and labels to NumPy arrays
print("[INFO] processing data...")
data = np.array(data, dtype="float32")
labels = np.array(labels)
 
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

[INFO] processing data...


In [9]:
# partition the data into training and testing splits
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=TEST_SPLIT, random_state=42)
# take the validation split from the training split
(trainX, valX, trainY, valY) = train_test_split(trainX, trainY,
    test_size=VAL_SPLIT, random_state=84)
# initialize the training data augmentation object
aug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [10]:
# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = VGG16(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(CLASSES), activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False
# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")

keras_callbacks = [EarlyStopping(monitor='val_loss',patience=3,mode='min',min_delta=0.0001),ModelCheckpoint('./Output',monitor='val_loss',save_best_only=True,mode='min')]
opt = SGD(learning_rate=MAX_LR, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])


[INFO] compiling model...


In [12]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
    validation_data=(valX, valY),
    steps_per_epoch=trainX.shape[0] // BATCH_SIZE,
    epochs=NUM_EPOCHS,
    verbose=1)

[INFO] training network...
Epoch 1/30
168/168 [==============================] - 1173s 7s/step - loss: 4.3534 - accuracy: 0.2975 - val_loss: 1.4482 - val_accuracy: 0.5200
Epoch 2/30
168/168 [==============================] - 1157s 7s/step - loss: 1.8327 - accuracy: 0.4250 - val_loss: 1.0063 - val_accuracy: 0.6883
Epoch 3/30
168/168 [==============================] - 1143s 7s/step - loss: 1.4705 - accuracy: 0.5171 - val_loss: 0.8413 - val_accuracy: 0.7267
Epoch 4/30
168/168 [==============================] - 1135s 7s/step - loss: 1.3090 - accuracy: 0.5574 - val_loss: 0.6823 - val_accuracy: 0.7883
Epoch 5/30
168/168 [==============================] - 1144s 7s/step - loss: 1.1679 - accuracy: 0.6075 - val_loss: 0.5864 - val_accuracy: 0.8033
Epoch 6/30
168/168 [==============================] - 1141s 7s/step - loss: 1.0138 - accuracy: 0.6461 - val_loss: 0.4959 - val_accuracy: 0.8283
Epoch 7/30
168/168 [==============================] - 1139s 7s/step - loss: 0.9714 - accuracy: 0.6716 - val_l

In [13]:
# evaluate the network and show a classification report
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BATCH_SIZE)
print(classification_report(testY.argmax(axis=1),
    predictions.argmax(axis=1), target_names=CLASSES))
# serialize the model to disk
print("[INFO] serializing network to '{}'...".format(MODEL_PATH))
model.save(MODEL_PATH)

[INFO] evaluating network...
              precision    recall  f1-score   support

      Card_0       1.00      1.00      1.00       112
      Card_1       1.00      0.98      0.99        91
      Card_2       0.93      0.90      0.91       107
      Card_3       0.99      0.91      0.95       123
      Card_4       0.92      0.90      0.91        94
      Card_5       0.97      0.96      0.97       103
      Card_6       0.92      0.91      0.91        95
      Card_7       0.90      1.00      0.95        88
      Card_8       0.87      0.98      0.92       104
      Card_9       1.00      0.98      0.99       102
     Draw_2+       0.96      0.96      0.96       104
     Draw_4+       1.00      1.00      1.00        91
     Reverse       1.00      0.97      0.99       104
   Skip_Card       0.97      1.00      0.98        87
   Wild_Card       1.00      1.00      1.00        95

    accuracy                           0.96      1500
   macro avg       0.96      0.96      0.96      15

In [14]:
# construct a plot that plots and saves the training history
N = np.arange(0,NUM_EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(TRAINING_PLOT_PATH)


In [4]:
# initialize the video stream, pointer to output video file, and
# frame dimensions
from collections import deque
print("[INFO] processing video...")
path ="C:/Users/97150/Desktop/Uno_card_color_shape_detection/Card_0.mov"
vs = cv2.VideoCapture(path)
# initialize the predictions queue
Q = []

(W, H) = (None, None)

# loop over frames from the video file stream
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()
 
    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break
 
    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        # clone the output frame, then convert it from BGR to RGB
        # ordering and resize the frame to a fixed 224x224
        output = frame.copy()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (224, 224))
        frame = frame.astype("float32")

        # make predictions on the frame and then update the predictions
        # queue
        #preds = model.predict(frame)
        preds = model.predict(np.expand_dims(frame, axis=0))
        Q.append(preds)
        # perform prediction averaging over the current history of
        # previous predictions
        results = Q
        i = np.argmax(results)
        label = CLASSES[i]

        # draw the activity on the output frame
        text = "detected card: {}".format(label)
        cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
            1.25, (0, 255, 0), 5)

        cv2.imshow("Output", output)
        key = cv2.waitKey(1) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

# release the file pointers
print("[INFO] cleaning up...")
vs.release()


[INFO] processing video...


NameError: name 'model' is not defined